In [ ]:
pip install -q tflite-model-maker

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [ ]:
from PIL import Image

In [ ]:
import os
import shutil 
# source = '/content/drive/MyDrive/ActivityRecognitionOfAnimals/test'
# source = '/content/drive/MyDrive/ActivityRecognitionOfAnimals/valid'
source = '/content/drive/MyDrive/ActivityRecognitionOfAnimals/train'

destination = '/content/sample_data/TempAnimalClassification'

dest = shutil.copytree(source, destination)

In [ ]:
classes_list = ["bear","elephant","leopard","lion","wolf","horse","zebra","tiger","fox","kangaroo","rhino","giraffe","bull","deer","crocodile"]

In [ ]:
import glob, os, random
for c in classes_list:
  os.chdir("/content/sample_data/AnimalClassification")
  if os.path.isdir(c) is False:
    os.makedirs(c)
  i = 0
  for filename in glob.iglob(os.path.join("/content/sample_data/TempAnimalClassification/"+c, '*')):
	  i+=1
  os.chdir("/content/sample_data/TempAnimalClassification/"+c)
  for r in random.sample(glob.glob('*'), i):
    shutil.move(r, "/content/sample_data/AnimalClassification/"+c)

In [ ]:
image_path = "/content/sample_data/AnimalClassification"
data = DataLoader.from_folder(image_path)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

INFO:tensorflow:Load image with size: 16168, num_label: 15, labels: bear, bull, crocodile, deer, elephant, fox, giraffe, horse, kangaroo, leopard, lion, rhino, tiger, wolf, zebra.


INFO:tensorflow:Load image with size: 16168, num_label: 15, labels: bear, bull, crocodile, deer, elephant, fox, giraffe, horse, kangaroo, leopard, lion, rhino, tiger, wolf, zebra.


In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

In [ ]:
model = image_classifier.create(train_data, model_spec=model_spec.get('mobilenet_v2'), validation_data=validation_data)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_2 (HubK (None, 1280)              2257984   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                19215     
Total params: 2,277,199
Trainable params: 19,215
Non-trainable params: 2,257,984
_________________________________________________________________
None
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


404/404 [==============================] - 1031s 3s/step - loss: 0.8818 - accuracy: 0.9001 - val_loss: 0.7366 - val_accuracy: 0.9556
Epoch 2/5
404/404 [==============================] - 993s 2s/step - loss: 0.7336 - accuracy: 0.9570 - val_loss: 0.7197 - val_accuracy: 0.9613
Epoch 3/5
404/404 [==============================] - 960s 2s/step - loss: 0.7116 - accuracy: 0.9650 - val_loss: 0.7129 - val_accuracy: 0.9638
Epoch 4/5
404/404 [==============================] - 953s 2s/step - loss: 0.6980 - accuracy: 0.9708 - val_loss: 0.7072 - val_accuracy: 0.9631
Epoch 5/5
404/404 [==============================] - 956s 2s/step - loss: 0.6903 - accuracy: 0.9756 - val_loss: 0.7034 - val_accuracy: 0.9625


In [ ]:
loss, accuracy = model.evaluate(test_data)

51/51 [==============================] - 300s 1s/step - loss: 0.7029 - accuracy: 0.9629


In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
model.export(export_dir='.')

INFO:tensorflow:Assets written to: /tmp/tmpdat1of25/assets


INFO:tensorflow:Assets written to: /tmp/tmpdat1of25/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpr4_cvd2j/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpr4_cvd2j/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


In [ ]:
model.evaluate_tflite('model.tflite', test_data)

{'accuracy': 0.9622758194186766}

In [ ]:
import os 
  
# importing shutil module 
import shutil 
# Source path 
source = '/tmp/model.tflite'
  
# Destination path 
destination = '/content/drive/MyDrive/ActivityRecognitionOfAnimals/Models/TF_Models/01_06_2021'
  
# Move the content of 
# source to destination 
dest = shutil.move(source, destination) 

In [ ]:
z=[]
import os,glob
for filename in glob.iglob(os.path.join("/content/sample_data/TempAnimalClassificationfo", '*')):
  image = Image.open(filename)
  try:
    new_image = image.resize((400, 400))
    new_image.save(filename)
  except:
    z.append(filename)